In [1]:
from magnet.base import Magnet
from magnet.base import EmbeddedMagnet

cluster = EmbeddedMagnet()
cluster.start()
magnet = cluster.create_magnet()
# await magnet.align()

config = {
    "host": "localhost",
    "credentials": None,
    "domain": None,
    "stream_name": "my_stream",
    "category": "my_category",
    "kv_name": "my_kv",
    "session": "my_session",
    "os_name": "my_object_store",
    "index": {
        "milvus_uri": "127.0.0.1",
        "milvus_port": 19530,
        "milvus_user": "test",
        "milvus_password": "test",
        "dimension": 1024,
        "model": "BAAI/bge-large-en-v1.5",
        "name": "test",
        "options": {
            'metric_type': 'COSINE',
            'index_type':'HNSW',
            'params': {
                "efConstruction": 40
                , "M": 48
            }
        }
    }
}

# magnet = Magnet(config)
await magnet.align()

🌊 SUCCESS: containerization engine connected
☠️ FATAL: 409 Client Error for http+docker://localhost/v1.44/containers/create?name=magnet-embedded-nats: Conflict ("Conflict. The container name "/magnet-embedded-nats" is already in use by container "c1f27b2e634e61e87aeedfd55711ef678e2c3f0bc9cd04fa8ead232fe418f126". You have to remove (or rename) that container to be able to reuse that name.")
☕️ WAIT: creating magnet with embedded cluster
🌊 SUCCESS: 🧲 connected to 
💎 nats://127.0.0.1:4222 
ℹ️ INFO: 🧲 initialized client 
🌊 SUCCESS: 🧲 connected to 
💎 MagnetConfig(host='127.0.0.1', domain=None, credentials=None, session='my_session', stream_name='my_stream', category='my_category', kv_name='my_kv', os_name='my_object_store', index=IndexConfig(milvus_uri='127.0.0.1', milvus_port=19530, milvus_user='test', milvus_password='test', dimension=1024, model='BAAI/bge-large-en-v1.5', name='test', options={'metric_type': 'COSINE', 'index_type': 'HNSW', 'params': {'efConstruction': 40, 'M': 48}})) 


In [2]:
from magnet.ic.field import Charge

field = Charge(magnet)
await field.on()

🌊 SUCCESS: ready [my_category] on
🛰️ stream: my_stream


In [3]:
# simple data pipelines with predictable outcomes
from magnet.utils.data_classes import FilePayload, Payload
from tractor_beam.utils.file_handlers import PDFProcessor
import base64

file = "/Users/dylan/VSCode/FinPDF/taking-stock-quarterly-outlook-en-us.pdf_2024-07-04.pdf"

with open(file, "rb") as fh:
    content = fh.read()
    encoded_content = base64.b64encode(content).decode('utf-8')

await field.pulse(
    FilePayload(encoded_content, file.split('/')[-1])
    , v=True
)

from magnet.ic.field import Resonator

reso = Resonator(magnet)

async def handle_payload(payload, msg):
    await reso.download(msg)
    processor = PDFProcessor(f"./{msg.name}.{msg.headers['ext']}")
    models = processor.load_models()
    file = await processor.export_to_markdown(f"./", msg.name, models)
    with open(f"{file}/{file.split('/')[-1]}.md", "r") as md:
        content = md.read()
        await field.pulse(Payload(content, file.split('/')[-1]), v=True)
    await payload.delete(msg.name)

await reso.on(obj=True)

await reso.listen(cb=handle_payload, v=True)

🌊 SUCCESS: uploaded to NATS object store in bucket my_object_store as d9dcecfcfbf607ee
☕️ WAIT: connecting to 127.0.0.1
ℹ️ INFO: subscribed to object store: my_object_store as Mac
ℹ️ INFO: consuming objects from [my_object_store] on
🛰️ stream: my_stream
🧲 session: "my_session"
🌊 SUCCESS: downloaded d9dcecfcfbf607ee from my_object_store


/opt/homebrew/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded detection model vikp/surya_det2 on device cpu with dtype torch.float32
Loaded detection model vikp/surya_layout2 on device cpu with dtype torch.float32
Loaded reading order model vikp/surya_order on device mps with dtype torch.float16
Loaded recognition model vikp/surya_rec on device mps with dtype torch.float16
Loaded texify model to mps with torch.float16 dtype


CoreGraphics PDF has logged an error. Set environment variable "CG_PDF_VERBOSE" to learn more.
Finding reading order: 100%|██████████| 1/1 [00:04<00:00,  4.72s/it]


🌊 SUCCESS: pulsed to my_category on my_stream


CancelledError: 

In [4]:
cluster.stop()
cluster.cleanup()

☕️ WAIT: stopping magnet-embedded-nats
🌊 SUCCESS: magnet-embedded-nats stopped
ℹ️ INFO: removing magnet-embedded-nats
🌊 SUCCESS: embedded nats removed
🌊 SUCCESS: embedded milvus server stopped
🚨 WARN: container engine pruned
🌊 SUCCESS: embedded cluster cleaned up
